<a href="https://colab.research.google.com/github/SamManuJacob/BIS_lab_1BM23CS291/blob/main/Lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# Objective function
def objective(x):
    return np.sum(x**2)  # Example (Sphere); replace with your function

# Levy flight
def levy_flight(Lambda):
    u = np.random.randn() * 0.01
    v = np.random.randn()
    step = u / (abs(v)**(1/Lambda))
    return step

def cuckoo_search(n=20, dim=2, lb=-10, ub=10, pa=0.25, max_iter=200):
    nests = np.random.uniform(lb, ub, (n, dim))
    fitness = np.array([objective(x) for x in nests])

    for _ in range(max_iter):
        # Generate new cuckoo by levy flights
        cuckoo = nests[np.random.randint(n)] + levy_flight(1.5) * np.random.randn(dim)
        cuckoo = np.clip(cuckoo, lb, ub)
        f_cuckoo = objective(cuckoo)

        # Random nest to compare
        j = np.random.randint(n)
        if f_cuckoo < fitness[j]:
            nests[j] = cuckoo
            fitness[j] = f_cuckoo

        # Abandon worst nests
        abandon = np.random.rand(n, dim) < pa
        steps = np.random.randn(n, dim) * (nests[np.random.permutation(n)] - nests)
        new_nests = nests + abandon * steps
        new_nests = np.clip(new_nests, lb, ub)

        new_fitness = np.array([objective(x) for x in new_nests])

        # Replace improved ones
        for i in range(n):
            if new_fitness[i] < fitness[i]:
                nests[i] = new_nests[i]
                fitness[i] = new_fitness[i]

    best_idx = np.argmin(fitness)
    return nests[best_idx], fitness[best_idx]


best_cs, fit_cs = cuckoo_search()
print("Cuckoo Search Best Solution:", best_cs)
print("Fitness:", fit_cs)


Cuckoo Search Best Solution: [ 3.09160378e-07 -4.98318206e-07]
Fitness: 3.439011738828179e-13


In [2]:
import numpy as np

def objective(x):
    return np.sum(x**2)  # Example (Sphere); replace with your function

def gwo(n=20, dim=2, lb=-10, ub=10, max_iter=200):
    wolves = np.random.uniform(lb, ub, (n, dim))
    fitness = np.array([objective(w) for w in wolves])

    alpha, beta, delta = np.zeros(dim), np.zeros(dim), np.zeros(dim)
    alpha_score, beta_score, delta_score = np.inf, np.inf, np.inf

    for _ in range(max_iter):
        for i, wolf in enumerate(wolves):
            score = fitness[i]

            if score < alpha_score:
                alpha_score, alpha = score, wolf.copy()
            elif score < beta_score:
                beta_score, beta = score, wolf.copy()
            elif score < delta_score:
                delta_score, delta = score, wolf.copy()

        a = 2 - _ * (2/max_iter)

        for i in range(n):
            for leader, leader_pos in zip([alpha, beta, delta], [alpha, beta, delta]):
                r1, r2 = np.random.rand(), np.random.rand()
                A = 2*a*r1 - a
                C = 2*r2
                D = abs(C*leader_pos - wolves[i])
                X = leader_pos - A*D

                if leader is alpha:
                    X1 = X
                elif leader is beta:
                    X2 = X
                else:
                    X3 = X

            wolves[i] = (X1 + X2 + X3)/3

        wolves = np.clip(wolves, lb, ub)
        fitness = np.array([objective(w) for w in wolves])

    return alpha, alpha_score


best_gwo, fit_gwo = gwo()
print("Grey Wolf Best Solution:", best_gwo)
print("Fitness:", fit_gwo)


Grey Wolf Best Solution: [ 1.75193350e-19 -1.73965955e-19]
Fitness: 6.095686323857123e-38
